In [1]:
# Erasmus+ ICCT project (2018-1-SI01-KA203-047081)

# Toggle cell visibility

from IPython.display import HTML
tag = HTML('''<script>
code_show=true; 
function code_toggle() {
    if (code_show){
        $('div.input').hide()
    } else {
        $('div.input').show()
    }
    code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
Promijeni vidljivost <a href="javascript:code_toggle()">ovdje</a>.''')
display(tag)

# Hide the code completely

# from IPython.display import HTML
# tag = HTML('''<style>
# div.input {
#     display:none;
# }
# </style>''')
# display(tag)


## Upravljanje povratnom vezom stanja za sustav masa-opruga-prigušivač

S obzirom na sustav masa-opruga-prigušivač, želimo ga upravljati na način da imamo step-odziv s nultom pogreškom u stacionarnom stanju i s vremenom smirivanja na 5% tolerantnog pojasa manjem od 6 s.

Jednadžbe sustava zapisane u formi prostora stanja su:


$$
\begin{bmatrix}
\dot{x_1} \\
\dot{x_2}
\end{bmatrix}=\underbrace{\begin{bmatrix}
0 && 1 \\
-\frac{k}{m} && -\frac{c}{m}
\end{bmatrix}}_{A}\begin{bmatrix}
x_1 \\
x_2
\end{bmatrix}+\underbrace{\begin{bmatrix}
0 \\
\frac{1}{m}
\end{bmatrix}}_{B}u,
$$

uz $m=5$ kg, $k=2$ N/m, $c=1$ Ns/m, $x_1$ predstavlja položaj, a $x_2$ brzinu. Definiranjem matrice pojačanja za povratnu vezu stanja kao $K=\begin{bmatrix}k_1&k_2\end{bmatrix}^T$ i supstitucijom $A-BK$ dobivamo:


$$
A-BK = \begin{bmatrix}0&1\\-\frac{2}{5}-\frac{k_1}{5}&-\frac{1}{5}-\frac{k_2}{5}\end{bmatrix}\,.
$$

Imajte na umu da je sustav u kanonskom obliku upravljivosti, stoga je karakteristični polinom

$$
\lambda^2+(\frac{k_2}{5}+\frac{1}{5})\lambda+(\frac{k_1}{5}+\frac{2}{5})
$$

Izjednačavanjem korijena s $\lambda_{1,2}=-1$ rad/s $\left((\lambda+1)^2=\lambda^2+2\lambda+1\right)$ dobivamo vrijednosti $k_1 = 3$ i $k_2=9$.

Da bi se postigla nulta pogreška u stacionarnom stanju, moguće je jednostavno prilagoditi pojačanje zatvorene petlje: pomnožimo referentni ulaz $u_{ref}$ s inverzom pojačanja zatvorene petlje kako bismo imali prijenosnu funkciju zatvorene petlje koja ostaje na $0$ dB pri niskim frekvencijama.

Statičko pojačanje izračunava se kao $G(0)=C(-A+BK)^{-1}B$ uz $C=\begin{bmatrix}1&0\end{bmatrix}$.

Konačni kontrolirani sustav, koji je još uvijek SISO od ulaza $u_{ref}$ do položaja $x_1$, je:

$$
\begin{cases}
\begin{bmatrix}
\dot{x_1} \\
\dot{x_2}
\end{bmatrix}=\underbrace{\begin{bmatrix}
0 && 1 \\
-1 && -2
\end{bmatrix}}_{A-BK}\begin{bmatrix}
x_1 \\
x_2
\end{bmatrix}+\underbrace{\begin{bmatrix}
0 \\
\frac{1}{5}
\end{bmatrix}}_{B}\frac{1}{0.2}u_{\text{ref}} \\
y = \begin{bmatrix}1&0\end{bmatrix}\begin{bmatrix}
x_1 \\
x_2
\end{bmatrix}
\end{cases}
$$

### Kako koristiti ovaj interaktivni primjer?
Pokušajte promijeniti svojstvene vrijednosti i prilagoditi pojačanje referentnog signala kako biste postigli nultu pogrešku u stacionarnom stanju.

In [2]:
%matplotlib inline
import control as control
import numpy
import sympy as sym
from IPython.display import display, Markdown
import ipywidgets as widgets
import matplotlib.pyplot as plt


#print a matrix latex-like
def bmatrix(a):
     """Returns a LaTeX bmatrix - by Damir Arbula (ICCT project)

     :a: numpy array
     :returns: LaTeX bmatrix as a string
     """
     if len(a.shape) > 2:
         raise ValueError('bmatrix can at most display two dimensions')
     lines = str(a).replace('[', '').replace(']', '').splitlines()
     rv = [r'\begin{bmatrix}']
     rv += ['  ' + ' & '.join(l.split()) + r'\\' for l in lines]
     rv +=  [r'\end{bmatrix}']
     return '\n'.join(rv)


# Display formatted matrix: 
def vmatrix(a):
    if len(a.shape) > 2:
         raise ValueError('bmatrix can at most display two dimensions')
    lines = str(a).replace('[', '').replace(']', '').splitlines()
    rv = [r'\begin{vmatrix}']
    rv += ['  ' + ' & '.join(l.split()) + r'\\' for l in lines]
    rv +=  [r'\end{vmatrix}']
    return '\n'.join(rv)


#matrixWidget is a matrix looking widget built with a VBox of HBox(es) that returns a numPy array as value !
class matrixWidget(widgets.VBox):
    def updateM(self,change):
        for irow in range(0,self.n):
            for icol in range(0,self.m):
                self.M_[irow,icol] = self.children[irow].children[icol].value
                #print(self.M_[irow,icol])
        self.value = self.M_

    def dummychangecallback(self,change):
        pass
    
    
    def __init__(self,n,m):
        self.n = n
        self.m = m
        self.M_ = numpy.matrix(numpy.zeros((self.n,self.m)))
        self.value = self.M_
        widgets.VBox.__init__(self,
                             children = [
                                 widgets.HBox(children = 
                                              [widgets.FloatText(value=0.0, layout=widgets.Layout(width='90px')) for i in range(m)]
                                             ) 
                                 for j in range(n)
                             ])
        
        #fill in widgets and tell interact to call updateM each time a children changes value
        for irow in range(0,self.n):
            for icol in range(0,self.m):
                self.children[irow].children[icol].value = self.M_[irow,icol]
                self.children[irow].children[icol].observe(self.updateM, names='value')
        #value = Unicode('example@example.com', help="The email value.").tag(sync=True)
        self.observe(self.updateM, names='value', type= 'All')
        
    def setM(self, newM):
        #disable callbacks, change values, and reenable
        self.unobserve(self.updateM, names='value', type= 'All')
        for irow in range(0,self.n):
            for icol in range(0,self.m):
                self.children[irow].children[icol].unobserve(self.updateM, names='value')
        self.M_ = newM
        self.value = self.M_
        for irow in range(0,self.n):
            for icol in range(0,self.m):
                self.children[irow].children[icol].value = self.M_[irow,icol]
        for irow in range(0,self.n):
            for icol in range(0,self.m):
                self.children[irow].children[icol].observe(self.updateM, names='value')
        self.observe(self.updateM, names='value', type= 'All')        

                #self.children[irow].children[icol].observe(self.updateM, names='value')

             
#overlaod class for state space systems that DO NOT remove "useless" states (what "professor" of automatic control would do this?)
class sss(control.StateSpace):
    def __init__(self,*args):
        #call base class init constructor
        control.StateSpace.__init__(self,*args)
    #disable function below in base class
    def _remove_useless_states(self):
        pass

In [3]:
# Preparatory cell

A = numpy.matrix([[0,1],[-2/5,-1/5]])
B = numpy.matrix('0; 1')
C = numpy.matrix('1 0')
X0 = numpy.matrix('0; 0')
K = numpy.matrix([3,9])

Aw = matrixWidget(2,2)
Aw.setM(A)
Bw = matrixWidget(2,1)
Bw.setM(B)
Cw = matrixWidget(1,2)
Cw.setM(C)
X0w = matrixWidget(2,1)
X0w.setM(X0)
Kw = matrixWidget(1,2)
Kw.setM(K)


eig1c = matrixWidget(1,1)
eig2c = matrixWidget(2,1)
eig1c.setM(numpy.matrix([-1])) 
eig2c.setM(numpy.matrix([[-1],[0]]))

In [4]:
# Misc

#create dummy widget 
DW = widgets.FloatText(layout=widgets.Layout(width='0px', height='0px'))

#create button widget
START = widgets.Button(
    description='Test',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Test',
    icon='check'
)
                       
def on_start_button_clicked(b):
    #This is a workaround to have intreactive_output call the callback:
    #   force the value of the dummy widget to change
    if DW.value> 0 :
        DW.value = -1
    else: 
        DW.value = 1
    pass
START.on_click(on_start_button_clicked)

# Define type of method 
selm = widgets.Dropdown(
    options= ['Postavi K', 'Postavi svojstvene vrijednosti'],
    value= 'Postavi svojstvene vrijednosti',
    description='',
    disabled=False
)

# Define the number of complex eigenvalues for the observer
selc = widgets.Dropdown(
    options= ['0 kompleksnih svojstvenih vrijednosti', '2 kompleksne svojstvene vrijednosti'],
    value= '0 kompleksnih svojstvenih vrijednosti',
    description='Svojstvene vrijednosti:',
    disabled=False
)

#define type of ipout 
selu = widgets.Dropdown(
    options=['impuls', 'step', 'sinus', 'Pravokutni val'],
    value='step',
    description='Tip referentnog signala:',
    style = {'description_width': 'initial'},
    disabled=False
)
# Define the values of the input
u = widgets.FloatSlider(
    value=1,
    min=0,
    max=20.0,
    step=0.1,
    description='Referentni ulaz:',
    style = {'description_width': 'initial'},
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',
)
period = widgets.FloatSlider(
    value=1,
    min=0.01,
    max=4,
    step=0.01,
    description='Period: ',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.2f',
)
gain = widgets.FloatText(
    value=0.2,
    description='Inverzno referentno pojačanje:',
    style = {'description_width': 'initial'},
    disabled=False
)
m = widgets.FloatSlider(
    value=5,
    min=0.1,
    max=10.0,
    step=0.1,
    description='m [kg]:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',
)
k = widgets.FloatSlider(
    value=2,
    min=0,
    max=10.0,
    step=0.1,
    description='k [N/m]:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',
)
c = widgets.FloatSlider(
    value=1,
    min=0,
    max=10.0,
    step=0.1,
    description='c [Ns/m]:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',
)

In [5]:
# Support functions

def eigen_choice(selc):
    if selc == '0 kompleksnih svojstvenih vrijednosti':
        eig1c.children[0].children[0].disabled = False
        eig2c.children[1].children[0].disabled = True
        eigc = 0
    if selc == '2 kompleksne svojstvene vrijednosti':
        eig1c.children[0].children[0].disabled = True
        eig2c.children[1].children[0].disabled = False
        eigc = 2
    return eigc

def method_choice(selm):
    if selm == 'Postavi K':
        method = 1
        selc.disabled = True
    if selm == 'Postavi svojstvene vrijednosti':
        method = 2
        selc.disabled = False
    return method

In [6]:
def main_callback(m, k, c, gain, X0w, K, eig1c, eig2c, u, period, selm, selc, selu, DW):
    A, B = numpy.matrix([[0,1],[-k/m,-c/m]]), numpy.matrix([[0],[1/m]])
    sols = numpy.linalg.eig(A)
    eigc = eigen_choice(selc)
    method = method_choice(selm)
    
    if method == 1:
        sol = numpy.linalg.eig(A-B*K)
    if method == 2:
        if eigc == 0:
            K = control.acker(A, B, [eig1c[0,0], eig2c[0,0]])
            Kw.setM(K) 
        if eigc == 2:
            K = control.acker(A, B, [numpy.complex(eig2c[0,0],eig2c[1,0]), 
                                     numpy.complex(eig2c[0,0],-eig2c[1,0])])
            Kw.setM(K)
        sol = numpy.linalg.eig(A-B*K)
    print('Svojstvene vrijednosti sustava su:',round(sols[0][0],4),'and',round(sols[0][1],4))
    print('Svojstvene vrijednosti kontroliranog sustava su:',round(sol[0][0],4),'and',round(sol[0][1],4))
    
    sys1 = sss(A-B*K,B,C,0)
    sg = control.evalfr(sys1,0)
    print('Statičko pojačanje kontroliranog sustava je: %f' %sg)
    if gain != 0:
        sys = sss(A-B*K,B*1/gain,C,0)
    else:
        print('Inverzno pojačanje postavljeno na 0 i promijenjeno na 1')
        sys = sss(A-B*K,B,C,0)
    T = numpy.linspace(0, 10, 1000)
      
    if selu == 'impuls': #selu
        U = [0 for t in range(0,len(T))]
        U[0] = u
        T, yout, xout = control.forced_response(sys,T,U,X0w)
    if selu == 'step':
        U = [u for t in range(0,len(T))]
        T, yout, xout = control.forced_response(sys,T,U,X0w)
    if selu == 'sinus':
        U = u*numpy.sin(2*numpy.pi/period*T)
        T, yout, xout = control.forced_response(sys,T,U,X0w)
    if selu == 'Pravokutni val':
        U = u*numpy.sign(numpy.sin(2*numpy.pi/period*T))
        T, yout, xout = control.forced_response(sys,T,U,X0w)
    
    fig = plt.figure(num='Bodeov graf', figsize=(16,10))
    control.bode_plot(sys)
    fig.suptitle('Bodeov graf', fontsize=16)
    
    plt.figure(num='Simulacija', figsize=(16,4))
    plt.title('Odziv za ulaz pozicije')
    plt.ylabel('pozicija vs referentni signal')
    plt.plot(T,xout[0],T,U,'r--')
    plt.xlabel('$t$ [s]')
    plt.axvline(x=0,color='black',linewidth=0.8)
    plt.axhline(y=0,color='black',linewidth=0.8)
    plt.legend(['pozicija','Referentni signal'])
    plt.grid()

    
alltogether = widgets.VBox([widgets.HBox([selm, 
                                          selc, 
                                          selu]),
                            widgets.Label(' ',border=3),
                            widgets.HBox([widgets.Label('K:',border=3), Kw, 
                                          widgets.Label(' ',border=3),
                                          widgets.Label(' ',border=3),
                                          widgets.Label('Svojstvene vrijednosti:',border=3), 
                                          eig1c, 
                                          eig2c, 
                                          widgets.Label(' ',border=3),
                                          widgets.Label(' ',border=3),
                                          widgets.Label('X0:',border=3), X0w]),
                            widgets.Label(' ',border=3),
                            widgets.HBox([u, 
                                          period, 
                                          START]),
                            widgets.Label(' ',border=3),
                            widgets.HBox([m,
                                          k,
                                          c,
                                          gain])])
out = widgets.interactive_output(main_callback, {'m':m, 'k':k, 'c':c, 'gain':gain, 'X0w':X0w, 'K':Kw, 'eig1c':eig1c, 'eig2c':eig2c, 
                                                 'u':u, 'period':period, 'selm':selm, 'selc':selc, 'selu':selu, 'DW':DW})
out.layout.height = '1050px'
display(out, alltogether)

Output(layout=Layout(height='1050px'))